In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import os,gc
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc,roc_auc_score 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
torch.cuda.set_device(1)
print (torch.cuda.current_device())
#os.environ['CUDA_VISIBLE_DEVICES'] = "2,3,4,5,6,7"

1


In [2]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

class DatasetGenerator(Dataset):
    def __init__(self, path_to_img_dir, path_to_dataset_file, transform=None):
        """
        初始化PyTorch的Dataset类以使用dataloader类
        :param path_to_img_dir: 存储ChestX-ray14 Dataset的图片文件的路径
        :param path_to_dataset_file: 存储`test.txt`, `train.txt`等文件的路径（训练集、测试集划分及图片所属的标签）
        :param transform: 对图片要做的transform处理
        """
        self.list_image_paths = []
        self.list_image_labels = []
        self.transform = transform
        with open(path_to_dataset_file, "r") as file_descriptor:
            lines = file_descriptor.readlines()
            for line in lines:
                line_items = line.split()
                image_path = os.path.join(path_to_img_dir, line_items[0].split('/')[1])  # line_items为图片的相对路径信息
                image_label = line_items[1:]  # 从第二个开始，都为标签信息
                image_label = [int(i) for i in image_label]  # 通过list生成向量
                self.list_image_paths.append(image_path)
                if (np.array(image_label).sum()==0): #normal
                    self.list_image_labels.append([1,0])
                else: self.list_image_labels.append([0,1]) #disease

    def __getitem__(self, index):
        """
        
        :param index: get item 时提供的索引index数值
        :return:
            imageData: 图片数据
            imageLabel: 标签信息
        """
        image_path = self.list_image_paths[index]
        image_data = Image.open(image_path).convert('RGB')
        image_label = torch.FloatTensor(self.list_image_labels[index])

        if self.transform:
            image_data = self.transform(image_data)

        return image_data, image_label

    def __len__(self):
        """
        :return:
            len: 数据集的总长度
        """
        return len(self.list_image_paths)


PATH_TO_IMAGES_DIR = '/data/fjsdata/NIH-CXR/images/images/'
PATH_TO_TRAIN_FILE = '/data/fjsdata/NIH-CXR/chexnet_dataset/train.txt'
PATH_TO_VAL_FILE = '/data/fjsdata/NIH-CXR/chexnet_dataset/val.txt'
PATH_TO_TEST_FILE = '/data/fjsdata/NIH-CXR/chexnet_dataset/test.txt'
def get_train_dataloader(batch_size, shuffle, num_workers, transform_seq):
    dataset_train = DatasetGenerator(path_to_img_dir=PATH_TO_IMAGES_DIR,
                                     path_to_dataset_file=PATH_TO_TRAIN_FILE, transform=transform_seq)
    data_loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size,
                                   shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    return data_loader_train


def get_validation_dataloader(batch_size, shuffle, num_workers, transform_seq):
    dataset_validation = DatasetGenerator(path_to_img_dir=PATH_TO_IMAGES_DIR,
                                          path_to_dataset_file=PATH_TO_VAL_FILE, transform=transform_seq)
    data_loader_validation = DataLoader(dataset=dataset_validation, batch_size=batch_size,
                                   shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    return data_loader_validation


def get_test_dataloader(batch_size, shuffle, num_workers, transform_seq):
    dataset_test = DatasetGenerator(path_to_img_dir=PATH_TO_IMAGES_DIR,
                                    path_to_dataset_file=PATH_TO_TEST_FILE, transform=transform_seq)
    data_loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size,
                                   shuffle=shuffle, num_workers=num_workers, pin_memory=True)
    return data_loader_test

In [3]:
# construct model
class DenseNet121(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet121, self).__init__()
        self.dense_net_121 = torchvision.models.densenet121(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_121.classifier.in_features
        self.dense_net_121.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_121(x)
        return x


class DenseNet169(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet169, self).__init__()
        self.dense_net_169 = torchvision.models.densenet169(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_169.classifier.in_features
        self.dense_net_169.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_169(x)
        return x


class DenseNet201(nn.Module):
    def __init__(self, num_classes, is_pre_trained):
        super(DenseNet201, self).__init__()
        self.dense_net_201 = torchvision.models.densenet201(pretrained=is_pre_trained)
        num_fc_kernels = self.dense_net_201.classifier.in_features
        self.dense_net_201.classifier = nn.Sequential(nn.Linear(num_fc_kernels, num_classes), nn.Sigmoid())

    def forward(self, x):
        x = self.dense_net_201(x)
        return x

In [4]:
N_CLASSES = 2 #normal and abnormal classification
network_model = DenseNet121(num_classes=N_CLASSES, is_pre_trained=True).cuda()#initialize model
#network_model = torch.nn.DataParallel(network_model, device_ids=[0,1,2,3,4,5]).cuda()  # make model available multi GPU cores training
torch.backends.cudnn.benchmark = True  # improve train speed slightly
# normalize data with ImageNet mean and standard deviation
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# compose transform operations
transform_list = list()
transform_list.append(transforms.Resize(256))
transform_list.append(transforms.RandomResizedCrop(224))
transform_list.append(transforms.RandomHorizontalFlip())
transform_list.append(transforms.ToTensor())
transform_list.append(normalize)
transform_sequence = transforms.Compose(transform_list)
# get data loader object
dataloader_train = get_train_dataloader(batch_size=32, shuffle=True, num_workers=0, transform_seq=transform_sequence)
dataloader_val = get_validation_dataloader(batch_size=32,shuffle=False, num_workers=0, transform_seq=transform_sequence)

optimizer = torch.optim.Adam(network_model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, mode='min')
loss  = torch.nn.BCELoss()
# start training network
best_net, best_loss = None, float('inf')
for epoch_index in range(50):#max epoch = 1000
    network_model.train()  # set network as train mode
    with torch.autograd.enable_grad():
        for batch_index, (image, label) in enumerate(dataloader_train):
            label.cuda()
            var_image = torch.autograd.Variable(image).cuda()
            var_label = torch.autograd.Variable(label).cuda()
            var_output = network_model(var_image)

            loss_tensor = loss(var_output, var_label)
            optimizer.zero_grad()
            loss_tensor.backward()
            optimizer.step()
            sys.stdout.write('\r Epoch: {} / Step: {} : train loss = {}'.format(epoch_index+1, batch_index+1, float('%0.6f'%loss_tensor.item())))
            sys.stdout.flush()  

    # Validation Process
    network_model.eval()  # set network as eval mode without BN & Dropout
    with torch.autograd.no_grad():
        loss_val = 0.
        mean_loss_tensor = 0.
        for batch_index, (image, label) in enumerate(dataloader_val):
            label.cuda()
            var_image = torch.autograd.Variable(image).cuda()
            var_label = torch.autograd.Variable(label).cuda()
            var_output = network_model(var_image)

            curr_loss_tensor = loss(var_output, var_label)  # the output of loss() is a tensor
            mean_loss_tensor += curr_loss_tensor  # tensor op.
            loss_val += curr_loss_tensor.item()  # scalar op.

            sys.stdout.write('\r Epoch: {} / Step: {} : validation loss = {}'.format(epoch_index+1, batch_index+1, float('%0.6f'%curr_loss_tensor.item())))
            sys.stdout.flush()

    loss_val = loss_val / len(dataloader_val)  # scalar
    mean_loss_tensor = mean_loss_tensor / len(dataloader_val)  # tensor

    # End validation process
    scheduler.step(mean_loss_tensor.item())
    if loss_val < best_loss:
        best_loss = loss_val
        best_net = copy.deepcopy(network_model)        
print("best_loss = %.6f" % (best_loss))
network_model = network_model.cpu()#release gpu memory
torch.cuda.empty_cache()

 Epoch: 50 / Step: 351 : validation loss = 0.542297best_loss = 0.598165


In [5]:
def compute_auroc(ground_truth, prediction):
        out_auroc = []
        np_ground_truth = ground_truth.cpu().numpy()
        np_prediction = prediction.cpu().numpy()
        for i in range(N_CLASSES):
            # calculate the roc_auc_score of each class
            out_auroc.append(roc_auc_score(np_ground_truth[:, i], np_prediction[:, i]))
        return out_auroc
    
# normalize data with ImageNet mean and standard deviation
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# compose transform operations
transform_list = list()
transform_list.append(transforms.Resize(256))
transform_list.append(transforms.TenCrop(224))
transform_list.append(transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])))
transform_list.append(transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops])))
transform_sequence = transforms.Compose(transform_list)
# get test data loader
data_loader_test = get_test_dataloader(batch_size=32, shuffle=False, num_workers=0, transform_seq=transform_sequence)

# initialize test output with tensor of type CUDA-float
output_ground_truth = torch.FloatTensor().cuda()
output_prediction = torch.FloatTensor().cuda()

# start testing
best_net.eval()  # set network as eval mode without BN & Dropout
with torch.autograd.no_grad():
    for batch_index, (image, label) in enumerate(data_loader_test):
        label = label.cuda()
        output_ground_truth = torch.cat((output_ground_truth, label), 0)
        batch_size, n_crops, num_channels, height, width = image.size()
        var_image = torch.autograd.Variable(image.view(-1, num_channels, height, width).cuda())
        out = best_net(var_image)
        out_mean = out.view(batch_size, n_crops, -1).mean(1)
        output_prediction = torch.cat((output_prediction, out_mean.data), 0)
        sys.stdout.write('\r Step: {}'.format(batch_index+1))
        sys.stdout.flush()
    auroc_individual = compute_auroc(output_ground_truth, output_prediction)
    auroc_mean = np.array(auroc_individual).mean()

print('Mean AUROC is %f.' % auroc_mean)
CLASS_NAMES = ['Normal', 'Abnormal'] 
for i in range(len(auroc_individual)):
    print(CLASS_NAMES[i], auroc_individual[i])

 Step: 702Mean AUROC is 0.764562.
Normal 0.764564034213212
Abnormal 0.7645609457155434
